In [1]:
%pwd

'd:\\Work\\Chicken-Disease-Classification\\research'

In [2]:
import os

os.chdir("../")

In [3]:
%pwd

'd:\\Work\\Chicken-Disease-Classification'

## Creating entity

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
   root_dir: Path
   source_URL: str
   local_data_file: Path 
   unzip_dir: Path

## Create configuration manager

In [5]:
from chicken_disease.constants import *
from chicken_disease.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
            self,
            config_file_path=CONFIG_FILE_PATH,
            params_file_path=PARAMS_FILE_PATH
            ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

## Create components

In [7]:
import os
import urllib.request as requests
import zipfile
from chicken_disease import logger
from chicken_disease.utils.common import get_size

In [8]:
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            file_name , headers = requests.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
                )
            logger.info(f"{file_name} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
        
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,"r") as zip_ref:
            zip_ref.extractall(unzip_path)

## Create pipeline

In [9]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-11-18 14:42:40,108: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-11-18 14:42:40,111: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-18 14:42:40,112: INFO: common: created directory at: artifacts]
[2023-11-18 14:42:40,114: INFO: common: created directory at: artifacts/data_ingestion]
[2023-11-18 14:42:40,115: INFO: 42848312: File already exists of size: ~ 11345 KB]
